In [1]:
import pandas as pd
import numpy as np
import re
import joblib as jb
from scipy.stats import binom

In [2]:
like = jb.load('data_pkl/like.pkl.z')
tag_like_y = jb.load('data_pkl/tag_like_y.pkl.z')
tag_like_num = jb.load('data_pkl/tag_like_num.pkl.z')
tag_bias_factor = jb.load("data_pkl/tag_bias_factor.pkl.z")

# P-Value

In [3]:
tag_pvals = dict()
global_bias = 0.4093

for tag in tag_like_num.keys():
    k = tag_like_y[tag]
    n = tag_like_num[tag]
    tag_pvals[tag] = np.min([binom.cdf(k=k, n=n, p=global_bias), binom.sf(k=k-1, n=n, p=global_bias)])

# Bias Factor ponderado com p-value

In [4]:
def logit(pval, bias_factor=12):
    pval = np.max([pval, 1e-5])
    inverse_pval = 1 - pval
    logit_ = np.log(inverse_pval) - np.log(1 - inverse_pval)
    
    return np.max([0, logit_]) / bias_factor

In [5]:
tag_pvals_logit = dict()
for tag, pval in tag_pvals.items():
    tag_pvals_logit[tag] = logit(pval)

In [6]:
tag_bias_factor_weighted = dict()

for tag, bias in tag_bias_factor.items():
    tag_bias_factor_weighted[tag] = tag_bias_factor[tag] ** tag_pvals_logit[tag]

In [7]:
jb.dump(tag_pvals, 'data_pkl/tag_pvals.pkl.z')
jb.dump(tag_bias_factor_weighted, 'data_pkl/tag_bias_factor_weighted.pkl.z')

['data_pkl/tag_bias_factor_weighted.pkl.z']